IMPORTS PARA O PROGRAMA E NORMALIZAÇÃO DO TITLE


In [24]:
import pandas as pd
import re
df = pd.read_csv('data.csv')
if df['title'].dtype == 'object':
    df['title'] = df['title'].str.upper()
dfp = df[df['amount'] > 0]
print(dfp)
df_cf = pd.read_csv('categorias_financeiras_organizado.csv')
print(df_cf)
df_usr = pd.read_csv('categorias_financeiras_user.csv')
print(df_usr)
vl = pd.read_csv('valuesmax.csv')
print(vl)
#IMPORTACAO DO PANDAS E DAS LISTAS PYTHON

          date                                              title  amount
0   2025-08-30                                      DIVINO ETERNO    5.00
1   2025-08-30                             UBER UBER *TRIP HELP.U   15.58
2   2025-08-29                             UBER UBER *TRIP HELP.U    5.92
3   2025-08-28                                     JOAOPAULOELIAS   12.50
4   2025-08-27                               AMAZON PRIME ALUGUEL    6.90
5   2025-08-26                             UBER UBER *TRIP HELP.U    9.02
6   2025-08-24                               AMAZON PRIME ALUGUEL    4.90
8   2025-08-24                                    BURGER KING APP   75.80
9   2025-08-23                              IFD*56.119.929 YASMIN   17.00
10  2025-08-23                             UBER UBER *TRIP HELP.U    6.87
11  2025-08-23                                         UBER* TRIP    6.22
12  2025-08-23                                    PG *GAMERS CLUB   21.90
13  2025-08-22                        

CRIACAO DO PRIMEIRO MODULO (ADICAO DE FILTROS - INICIALMENTE PRE DEFINIDOS - E ESTIPULACAO DE METAS PARA GASTOS)

In [11]:
def adicionar_categoria_gastos():
    nome = input('Digite o nome do centro de custo desejado')
    titulo = input('Digite a palavra chave que identifica o titulo do gasto no extrato *EX: Corrida do Uber = Uber*')
    nome_padrao = nome.upper().lstrip().rstrip()
    titulo_padrao = titulo.upper().lstrip().rstrip()
    save = 'BOTAO SALVAR'
    cancel = 'BOTAO CANCELAR'

    if df_cf['CATEGORIA'].isin([nome_padrao]).any() or df_usr['CATEGORIA'].isin([nome_padrao]).any() :
        print('Essa categoria ja existe! Voce pode adicionar palavras chave para que seu gasto entre nas categorias padroes')
    elif nome == "" or titulo == "":
        print('Nao é possivel adicionar valores vazios')
    elif nome == "" and titulo == "":
        print('Nao é possivel adicionar valores vazios')
    else:
        nova_categoria = df_usr.loc[len(df_usr)] = [nome_padrao,titulo_padrao]
        print(nova_categoria)
        print(df_usr)
        df_usr.to_csv('categorias_financeiras_user.csv', index=False)

adicionar_categoria_gastos()


['TELEVISAO', 'PANASONIC']
    CATEGORIA     TITULO
3   TELEVISAO  PANASONIC
4          PC   TERABYTE
5  HOSPEDAGEM     AIRBNB


FILTRANDO CATEGORIAS (PRINTA FATURA SOMENTE COM AS CATEGORIAS PADRAO)

In [28]:
def filtro_categorias_default():
    filtros_padrao = df_cf['TITULO']
    regex_filtros_padrao = '|'.join(filtros_padrao.dropna().astype(str).map(re.escape))
    filtros_padroes = dfp[dfp['title'].str.contains(regex_filtros_padrao, case=False, na=False)]
    soma = (filtros_padroes[filtros_padroes['amount'] > 0]['amount'].sum())
    print(filtros_padroes)
    print(f'{soma:.2f}')

filtro_categorias_default()

          date                              title  amount
1   2025-08-30             UBER UBER *TRIP HELP.U   15.58
2   2025-08-29             UBER UBER *TRIP HELP.U    5.92
4   2025-08-27               AMAZON PRIME ALUGUEL    6.90
5   2025-08-26             UBER UBER *TRIP HELP.U    9.02
6   2025-08-24               AMAZON PRIME ALUGUEL    4.90
8   2025-08-24                    BURGER KING APP   75.80
9   2025-08-23              IFD*56.119.929 YASMIN   17.00
10  2025-08-23             UBER UBER *TRIP HELP.U    6.87
11  2025-08-23                         UBER* TRIP    6.22
13  2025-08-22                  IFD*WEILER BURGER   35.50
14  2025-08-21                         UBER* TRIP    7.65
15  2025-08-21             UBER UBER *TRIP HELP.U    7.04
17  2025-08-17             IFD*COMES E BEBES COME   10.00
21  2025-08-12                         DL*99 RIDE    7.90
22  2025-08-11                  NSA SUPERMERCADOS   24.99
24  2025-08-08  AIRBNB * HM58QNQA3C - PARCELA 1/3  127.68
25  2025-08-08

FILTRANDO GASTOS GERAIS (RETIRA OS GASTOS FILTRADOS E MOSTRA OS "SEM CATEGORIA")

In [27]:
def filtro_geral():
    filtros_padrao = df_cf['TITULO']
    regex_filtros_padrao = '|'.join(filtros_padrao.dropna().astype(str).map(re.escape))
    filtros_padroes = dfp[~(dfp['title'].str.contains(regex_filtros_padrao, case=False, na=False))]
    soma = (filtros_padroes[filtros_padroes['amount'] > 0]['amount'].sum())
    print(filtros_padroes)
    print(f'{soma:.2f}')

filtro_geral()

          date                                              title  amount
0   2025-08-30                                      DIVINO ETERNO    5.00
3   2025-08-28                                     JOAOPAULOELIAS   12.50
12  2025-08-23                                    PG *GAMERS CLUB   21.90
16  2025-08-20                                  57852747JOAOPAULO    7.00
18  2025-08-16                                     EMPORIO BARRIL   30.99
19  2025-08-14                                  57852747JOAOPAULO    7.00
20  2025-08-13                                  EBW*CANVA - NUPAY   35.00
23  2025-08-11                                         KING FRIOS   57.48
28  2025-08-06                51.519.881 VITOR GARC - PARCELA 2/2  158.55
29  2025-08-06  PARCELAMENTO DE COMPRA - PG *FALLEN COMERCIO E...   56.02
31  2025-08-06      PARCELAMENTO DE COMPRA - EMPORIO BARRIL - 2/2   23.21
414.65


BUSCANDO APENAS A CATEGORIA ESCOLHIDA

In [14]:
def pesquisar_categoria():
    categoria = input('Digite a categoria desejada: ').strip().upper()
    regra = df_cf[df_cf['CATEGORIA'] == categoria]
    regra_usr = df_usr[df_usr['CATEGORIA'] == categoria]
    if regra.empty and regra_usr.empty:
        print('CATEGORIA NAO ENCONTRADA')
        return
    regex = '|'.join(regra['TITULO'].dropna().astype(str).map(re.escape))
    regex_usr = '|'.join(regra_usr['TITULO'].dropna().astype(str).map(re.escape))
    if regra_usr.empty:
        resultado = dfp[dfp['title'].str.contains(regex,case=False,na=False)]
    if regra.empty:
        resultado = dfp[dfp['title'].str.contains(regex_usr,case=False,na=False)]
    if resultado.empty:
        print('Digite uma categoria válida')
        return
    print(resultado)
    soma = resultado['amount'].sum()
    print(soma)
pesquisar_categoria()

          date                   title  amount
8   2025-08-24         BURGER KING APP   75.80
9   2025-08-23   IFD*56.119.929 YASMIN   17.00
13  2025-08-22       IFD*WEILER BURGER   35.50
17  2025-08-17  IFD*COMES E BEBES COME   10.00
21  2025-08-12              DL*99 RIDE    7.90
26  2025-08-07          IFD*IFOOD CLUB    5.95
152.15


INSERINDO PALAVRAS CHAVE EM CATEGORIAS EXISTENTES

In [ ]:
def palavras_chave_categorias_dflt():
    categorias = input('Digite a categoria existente desejada:')
    categorias_padrao = categorias.upper().lstrip().rstrip()
    if df_cf['CATEGORIA'].isin([categorias_padrao]).any():
        titulo = input('Digite as palavras-chave que deseja incluir para reconhecimento do extrato')
        titulo_padrao = titulo.upper().lstrip().rstrip()
        nova_plchave = df_cf.loc[len(df_cf)] = [categorias_padrao,titulo_padrao]
        print(nova_plchave)
    else:
        print('Essa categoria nao existe! Voce pode criá-la ou incluir em alguma das categorias existentes')

palavras_chave_categorias_dflt()

ESTIPULANDO VALOR MAXIMO PARA CADA CATEGORIA

In [21]:
def valor_maximo_categoria():
    while True:
        categorias = input('Digite a categoria que deseja configurar:')
        categorias_padrao = categorias.upper().lstrip().rstrip()
        if df_cf['CATEGORIA'].isin([categorias_padrao]).any() or df_usr['CATEGORIA'].isin([categorias_padrao]).any() :
            valor = float(input('Digite o valor maximo que deseja gastar:'))
        else:
            print('Essa categoria nao existe!')
            return
        
        if vl['CATEGORIA'].isin([categorias_padrao]).any():
            print('Voce ja adicionou um valor para essa categoria')
            return
        add_valor = vl.loc[len(vl)] = [categorias_padrao,valor]
        print(add_valor)
        vl.to_csv('valuesmax.csv', index=False)
        return

valor_maximo_categoria()

['HOSPEDAGEM', 250.0]


VERIFICADOR DE VALOR POR CATEGORIA

In [55]:
def total_por_categoria():
    lista_cat = []
    lista_cat.extend(df_cf['CATEGORIA'].unique())
    lista_cat.extend(df_usr['CATEGORIA'].unique())
    for categoria in lista_cat:
        filtro = df_cf[df_cf['CATEGORIA'] == categoria]
        if filtro.empty:
            filtro = df_usr[df_usr['CATEGORIA'] == categoria]
        titulo = filtro['TITULO']
        if titulo.empty:
            continue
        regex_title = '|'.join(titulo.dropna().astype(str).map(re.escape))
        soma = dfp[dfp['title'].str.contains(regex_title, case=False, na=False)]['amount'].sum()
        val_max = vl.loc[vl['CATEGORIA'] == categoria, 'VALOR']
        if val_max.empty:
            print(f'{categoria} - sem limite definido')
            continue
        val_max = val_max.iloc[0]
        porcentagem = (soma / val_max) * 100 if val_max != 0 else 0
        print(f'{categoria} - {val_max:.2f} / {soma:.2f} = {porcentagem:.2f}%')
total_por_categoria()

DELIVERY - 500.00 / 152.15 = 30.43%
VAREJO - sem limite definido
TRANSPORTE - sem limite definido
MERCADO - 1000.00 / 24.99 = 2.50%
ENTRETENIMENTO - 250.00 / 11.80 = 4.72%
SAUDE - 500.00 / 0.00 = 0.00%
CONTAS - sem limite definido
HOSPEDAGEM - 250.00 / 127.68 = 51.07%
GABINETE - sem limite definido


RETIRANDO CATEGORIA DO USUARIO

In [26]:
def retirar_categoria():
    global df_usr
    while True:
        print(df_usr)
        categoria_usr = input('Qual categoria deseja retirar?').upper().lstrip().rstrip()
        if df_usr['CATEGORIA'].isin([categoria_usr]).any():
            print(df_usr[df_usr['CATEGORIA'] == categoria_usr])
        else:
            print('Essa categoria nao existe!')
            return
        titulo = input('Digite os titulos que voce deseja remover (caso queira remover toda a categoria, digite TODOS)').upper().lstrip().rstrip()
        if df_usr['TITULO'].isin([titulo]).any():
            index = (df_usr[df_usr['TITULO'] == titulo]).index
            df_usr = df_usr.drop(index)
            df_usr.to_csv('categorias_financeiras_user.csv', index=False)
            print(df_usr)
            return
        elif titulo == 'TODOS':
            index = (df_usr[df_usr['CATEGORIA'] == categoria_usr]).index
            df_usr = df_usr.drop(index)
            print(df_usr)
            return
        else:
            print('Esse titulo nao existe')
        df_usr.to_csv('categorias_financeiras_user.csv', index=False)
retirar_categoria()

  CATEGORIA TITULO
3     CARRO    BMW
  CATEGORIA TITULO
3     CARRO    BMW
Empty DataFrame
Columns: [CATEGORIA, TITULO]
Index: []


In [ ]:
print(dfp.sum())

In [ ]:
#COMO EU VOU SOMAR OS GASTOS POR FILTRO, PARA NO FINAL TIRAR A PORCENTAGEM EM RELACAO AO QUANTO EU GASTEI DESSAS CATEGORIAS COM O TOTAL QUE EU ESTIPULEI

'''
EXEMPLO:

COM DELIVERY, GASTEI 380, MAS O TOTAL QUE EU ESTIPULEI FOI DE 500 REAIS

-PRECISO PEGAR DO EXTRATO TODOS OS GASTOS QUE SE ENQUADRAM COM DELIVERY
-SOMAR A COLUNA DE AMOUNT
-DIVIDIR ESSE VALOR COM O TOTAL E MULTIPLICAR POR 100
-EM CIMA DO VALOR GERADO (PORCENTAGEM), UTILIZAR ESSE VALOR PRA INTERAGIR COM O FRONT E FAZER UM GRAFICO (DE 0-30 VERDE, DE 30-60 AMARELO, DE 60-100 VERMELHO)

'''